In [1]:
import skrub
import skrub.datasets
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier

from sempipes import *

In [2]:
dataset = skrub.datasets.fetch_credit_fraud()

products = skrub.var("products", dataset.products)
baskets = skrub.var("baskets", dataset.baskets)
baskets = baskets.skb.subsample(n=5000, how='random')

basket_ids = baskets[["ID"]].skb.mark_as_X()
fraud_flags = baskets["fraud_flag"].skb.mark_as_y()

In [3]:
products

<Var 'products'>
Result:
―――――――
        basket_ID                              item  cash_price         make  \
1           51113  COMPUTER PERIPHERALS ACCESSORIES         409        APPLE   
9           41798                         COMPUTERS        1187        APPLE   
11          39361                         COMPUTERS         898        APPLE   
15          38615  COMPUTER PERIPHERALS ACCESSORIES         379        APPLE   
16          70262                         COMPUTERS        1899        APPLE   
...           ...                               ...         ...          ...   
163352      42613                 BEDROOM FURNITURE         259  SILENTNIGHT   
163353      42613                 OUTDOOR FURNITURE         949   LG OUTDOOR   
163354      43567                         COMPUTERS        1099        APPLE   
163355      43567                         COMPUTERS        2099        APPLE   
163356      68268           TELEVISIONS HOME CINEMA         799           LG   

                                                    model goods_code  \
1       APPLE WATCH SERIES 6 GPS 44MM SPACE GREY ALUMI...  239001518   
9       2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESS...  239246780   
11      2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 ...  239246776   
15      APPLE WATCH SERIES 6 GPS 40MM BLUE ALUMINIUM CASE  239001540   
16      2021 APPLE MACBOOK PRO 14 M1 PRO PROCESSOR 16G...  240575990   
...                                                   ...        ...   
163352  SILENTNIGHT SLEEP GENIUS FULL HEIGHT HEADBOARD...  236938439   
163353  LG OUTDOOR BERGEN 2-SEAT GARDEN SIDE TABLE REC...  239742814   
163354  2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-F...  240040978   
163355  2020 APPLE IMAC 27 ALL-IN-ONE INTEL CORE I7 8G...  238923518   
163356  LG OLED48A16LA 2021 OLED HDR 4K ULTRA HD SMART TV  239866717   

        Nbr_of_prod_purchas  
1                         1  
9                         1  
11                        1  
15                        1  
16                        1  
...                     ...  
163352                    1  
163353                    1  
163354                    1  
163355                    1  
163356                    1  

[109380 rows x 7 columns]

In [4]:
products = products.sem_fillna("make", "Infer the manufacturer from product descriptions")
products

--- Sempipes.sem_fillna('make', 'Infer the manufacturer from product descriptions')
	Querying 'openai/gpt-5' with: 'The data scientist wants to fill missing values in the column 'make' of typ...'
    Training imputation model RandomForestClassifier(random_state=0) on columns ['item', 'model', 'goods_code'] of 108107 rows...
    Imputing 1273 values...


<Call 'sempipes_sem_fillna'>
Result:
―――――――
        basket_ID                              item  cash_price         make  \
1           51113  COMPUTER PERIPHERALS ACCESSORIES         409        APPLE   
9           41798                         COMPUTERS        1187        APPLE   
11          39361                         COMPUTERS         898        APPLE   
15          38615  COMPUTER PERIPHERALS ACCESSORIES         379        APPLE   
16          70262                         COMPUTERS        1899        APPLE   
...           ...                               ...         ...          ...   
163352      42613                 BEDROOM FURNITURE         259  SILENTNIGHT   
163353      42613                 OUTDOOR FURNITURE         949   LG OUTDOOR   
163354      43567                         COMPUTERS        1099        APPLE   
163355      43567                         COMPUTERS        2099        APPLE   
163356      68268           TELEVISIONS HOME CINEMA         799           LG   

                                                    model goods_code  \
1       APPLE WATCH SERIES 6 GPS 44MM SPACE GREY ALUMI...  239001518   
9       2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESS...  239246780   
11      2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 ...  239246776   
15      APPLE WATCH SERIES 6 GPS 40MM BLUE ALUMINIUM CASE  239001540   
16      2021 APPLE MACBOOK PRO 14 M1 PRO PROCESSOR 16G...  240575990   
...                                                   ...        ...   
163352  SILENTNIGHT SLEEP GENIUS FULL HEIGHT HEADBOARD...  236938439   
163353  LG OUTDOOR BERGEN 2-SEAT GARDEN SIDE TABLE REC...  239742814   
163354  2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-F...  240040978   
163355  2020 APPLE IMAC 27 ALL-IN-ONE INTEL CORE I7 8G...  238923518   
163356  LG OLED48A16LA 2021 OLED HDR 4K ULTRA HD SMART TV  239866717   

        Nbr_of_prod_purchas  
1                         1  
9                         1  
11                        1  
15                        1  
16                        1  
...                     ...  
163352                    1  
163353                    1  
163354                    1  
163355                    1  
163356                    1  

[109380 rows x 7 columns]

In [6]:
kept_products = products[products["basket_ID"].isin(basket_ids["ID"])]
kept_products = kept_products.with_sem_features("Compute additional brand-related product features", how_many=5)

--- Sempipes.with_sem_features('Compute additional brand-related product features', 5)
	Querying 'openai/gpt-5' with: 'The dataframe `df` is loaded and in memory. Columns are also named attributes....'
	Generated 5 feature columns: ['brand_freq_global', 'brand_is_apple', 'brand_is_retailer', 'brand_item_share', 'price_vs_brand_avg']


In [7]:
kept_products

<Call 'sempipes_with_sem_features'>
Result (on a subsample):
――――――――――――――――――――――――
        basket_ID                              item  cash_price      make  \
58          45079           LIVING DINING FURNITURE        1899  RETAILER   
82          53672                         COMPUTERS        1649     APPLE   
134         33017                         COMPUTERS         899     APPLE   
135         33017                          WARRANTY          70  RETAILER   
136         33017                   CABLES ADAPTERS          17     APPLE   
...           ...                               ...         ...       ...   
163325      32230  COMPUTER PERIPHERALS ACCESSORIES         165     APPLE   
163326      32230                         COMPUTERS        1149     APPLE   
163331      39512                         COMPUTERS        1649     APPLE   
163332      39512                 FULFILMENT CHARGE           7  RETAILER   
163350      45891                         COMPUTERS         898     APPLE   

                                                    model  goods_code  \
58                RETAILER DRAPER GRAND 4 SEATER SOFA LIG   238121946   
82      2021 APPLE IMAC 24 ALL-IN-ONE M1 PROCESSOR 8GB...   240040963   
134     2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 ...   239246782   
135                                              RETAILER   236604744   
136                  APPLE MJ1M2ZM A USB-C TO USB ADAPTOR   234418518   
...                                                   ...         ...   
163325             APPLE MAGIC KEYBOARD FOR 12 9 IPAD PRO   238701670   
163326  2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESS...   239246779   
163331  2021 APPLE IMAC 24 ALL-IN-ONE M1 PROCESSOR 8GB...   240040964   
163332                                           RETAILER  FULFILMENT   
163350  2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 ...   239246776   

        Nbr_of_prod_purchas  brand_is_apple  brand_is_retailer  \
58                        1               0                  1   
82                        1               1                  0   
134                       1               1                  0   
135                       1               0                  1   
136                       1               1                  0   
...                     ...             ...                ...   
163325                    1               1                  0   
163326                    1               1                  0   
163331                    1               1                  0   
163332                    1               0                  1   
163350                    1               1                  0   

        brand_freq_global  brand_item_share  price_vs_brand_avg  
58               0.325272          0.643333         1696.040894  
82               0.469603          0.984638          806.444580  
134              0.469603          0.984638           56.444603  
135              0.325272          1.000000         -132.959076  
136              0.469603          0.948718         -825.555420  
...                   ...               ...                 ...  
163325           0.469603          0.949250         -677.555420  
163326           0.469603          0.984638          306.444611  
163331           0.469603          0.984638          806.444580  
163332           0.325272          1.000000         -195.959076  
163350           0.469603          0.984638           55.444603  

[9014 rows x 12 columns]

In [8]:
vectorizer = skrub.TableVectorizer()

vectorized_products = kept_products.skb.apply_with_sem_choose(vectorizer, exclude_cols="basket_ID", 
    choices=sem_choose(high_cardinality="Different encoders for messy columns with potentially invalid data"))

aggregated_products = vectorized_products.groupby("basket_ID").agg("mean").reset_index()

augmented_baskets = basket_ids.merge(aggregated_products, left_on="ID", right_on="basket_ID").drop(columns=["ID", "basket_ID"])

--- Sempipes.apply_with_sem_choose(TableVectorizer(), {'high_cardinality': 'Different encoders for messy columns with potentially invalid data'})
	Querying 'openai/gpt-5' with: 'You need to help a data scientist improve their machine learning script in...'
An error occurred in attempt 1: SimilarityEncoder.__init__() got an unexpected keyword argument 'similarity'
Traceback (most recent call last):
  File "/Users/ssc/projects/sempipes/sempipes/_choose.py", line 112, in apply_with_sem_choose
    suggested_choices = _safe_exec(python_code, "__generated_sempipes_choices")
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ssc/projects/sempipes/sempipes/code_gen/_exec.py", line 46, in _safe_exec
    exec(python_code, safe_globals, safe_locals)
  File "<string>", line 12, in <module>
TypeError: SimilarityEncoder.__init__() got an unexpected keyword argument 'similarity'

	Querying 'openai/gpt-5' with: 'You need to help a data scientist improve thei

In [9]:
hgb = HistGradientBoostingClassifier()
predictions = augmented_baskets.skb.apply_with_sem_choose(hgb, y=fraud_flags, 
  choices=sem_choose(learning_rate="Three learning rates to try"))

--- Sempipes.apply_with_sem_choose(HistGradientBoostingClassifier(), {'learning_rate': 'Three learning rates to try'})
	Querying 'openai/gpt-5' with: 'You need to help a data scientist improve their machine learning script in...'
	Suggested choices for learning_rate: choose_from([0.01, 0.05, 0.1])


In [10]:
predictions

<Apply HistGradientBoostingClassifier>
Result (on a subsample):
――――――――――――――――――――――――
      fraud_flag
0              0
1              0
2              0
3              0
4              0
...          ...
4995           0
4996           0
4997           0
4998           0
4999           0

[5000 rows x 1 columns]

In [ ]:
#search = predictions.skb.make_randomized_search(scoring="roc_auc", n_iter=4, n_jobs=1, fitted=True, keep_subsampling=True, cv=2)
#search.results_